In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install einops


In [ ]:
!pip install tqdm scipy
!pip install trl
%pip install -U datasets

In [ ]:
# from kaggle_secrets import UserSecretsClient
# !pip install wandb
# import wandb
# user_secrets = UserSecretsClient()
# secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
# secret_wandb = user_secrets.get_secret("wandb")
# !huggingface-cli login --token $secret_hf
# wandb_api = user_secrets.get_secret("wandb")
# wandb.login(key=wandb_api)
# wandb.init(project="Phi-2_10k")

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install wandb
import wandb
wandb.login()
wandb.init(project="phi-2 10k")

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset


In [1]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

In [2]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model

In [3]:

# Accessing the train and validation datasets
dataset1 = load_dataset("Aneeth/job_description_10k", split="train")
dataset2 = load_dataset("Aneeth/job_description_10k", split="validation")


In [4]:
import pandas as pd
df1 = pd.DataFrame(dataset1)
df2 = pd.DataFrame(dataset2)

In [5]:
def format_row(row):
    question = row['user_prompt']
    answer = row['model_response']
    formatted_string = f"[INST] {question} [/INST] {answer} "
    return formatted_string

# Apply the function to each row of the dataframe
df1['Text'] = df1.apply(format_row, axis=1)
df2['Text'] = df2.apply(format_row, axis=1)




In [7]:
from datasets import Dataset
training_dataset = Dataset.from_pandas(df1)
eval_dataset = Dataset.from_pandas(df2)

In [8]:
base_model = "microsoft/phi-2"
new_model = "phi-2-job-desp"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    # use_flash_attention_2=True, # Phi does not support yet.
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    revision="refs/pr/23",
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
    output_dir="./Phi-2JD",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=50,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    save_steps=500,
    warmup_ratio=0.1,
    weight_decay=0.01,
    push_to_hub=True,  # Add this line to push the model to the hub,
)

peft_config = LoraConfig(
    r=64,
    lora_alpha=1,
    lora_dropout=0.5,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ], - 41M params
    # modules_to_save=["embed_tokens","lm_head"]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=3000,
    tokenizer=tokenizer,
    args=training_arguments,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
trainer.push_to_hub()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: Currently logged in as: aneeth-manikantan121. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,1.478600,1.415354
1000,0.799400,0.732064
1500,0.638700,0.587847
2000,0.572300,0.511572
2500,0.523200,0.460928
3000,0.490300,0.426337
3500,0.452400,0.406730
4000,0.444200,0.395734
4500,0.448400,0.391980


You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
You are using a model of type phi to instantiate a model of type phi-msft. This is not s

KeyboardInterrupt: 